In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from utils import *
from sklearn.metrics import roc_curve
from keras.callbacks import EarlyStopping
from keras import backend
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils_keras import KerasModelWrapper
import matplotlib as mpl
from keras.models import load_model

DATASET_NAME = 'cifar10'
(x_train, y_train, x_test, y_test) = get_dataset(DATASET_NAME)

Using TensorFlow backend.


x_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
x_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


In [21]:
def pixel2phase_before(images):
    img_fft = np.fft.fft2(images)
    phase = np.angle(img_fft)
    return phase

In [22]:
## Phase before Permutation + MLP   (seed 87)
SECRET_SEED = 87

x_train_phase = pixel2phase_before(x_train)
x_test_phase = pixel2phase_before(x_test)
x_train_shuffle = permute_pixels(x_train_phase, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test_phase, SECRET_SEED)

order_model = get_ppd_model(name=DATASET_NAME, ppd=False)
order_model.summary()
order_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

order_model.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_images (InputLayer)    (None, 32, 32, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense800 (Dense)             (None, 800)               2458400   
_________________________________________________________________
dense300 (Dense)             (None, 300)               240300    
_________________________________________________________________
dense10 (Dense)              (None, 10)                3010      
Total params: 2,701,710
Trainable params: 2,701,710
Non-trainable params: 0
_________________________________________________________________
Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 4s 73us/step - loss: 2.1

In [23]:
sess =  backend.get_session()
backend.set_learning_phase(False)
attack_params = {
    'eps': 0.4
    }

def get_adv_exapmles(model, x, sess, attack_params):
    wrap = KerasModelWrapper(model)
    attack_model = FastGradientMethod(wrap, sess=sess)
    x_adv = attack_model.generate_np(x, **attack_params)
    return x_adv

x_adv_order = get_adv_exapmles(order_model, x_test, sess, attack_params)


[INFO 2018-12-03 20:28:28,391 cleverhans] Constructing new graph for attack FastGradientMethod


In [24]:
def test_model(model, x, y):
    pred = np.argmax(model.predict(x), axis = 1)
    acc =  np.mean(np.equal(pred.reshape(y.shape), y))
    return (pred, acc)

x_test_phase = pixel2phase_before(x_test)
x_adv_phase = pixel2phase_before(x_adv_order)
x_shuffle = permute_pixels(x_test_phase, SECRET_SEED)
x_adv_shuffle = permute_pixels(x_adv_phase, SECRET_SEED)

pred, acc = test_model(order_model, x_shuffle, y_test)
print ('Order changed PPD MLP acc normally', acc)
pred, acc = test_model(order_model, x_adv_shuffle, y_test)
print ('Order changed PPD MLP acc adv', acc)


Order changed PPD MLP acc normally 0.2652
Order changed PPD MLP acc adv 0.1624


## Apply pixel2phase in hidden

In [25]:
## Normal PPD MLP   (seed 87)
SECRET_SEED = 87

x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

normal_model = get_ppd_model(name=DATASET_NAME, ppd=True)
normal_model.summary()
normal_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

normal_model.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_images (InputLayer)    (None, 32, 32, 3)         0         
_________________________________________________________________
pixel2phase (Lambda)         (None, 32, 32, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense800 (Dense)             (None, 800)               2458400   
_________________________________________________________________
dense300 (Dense)             (None, 300)               240300    
_________________________________________________________________
dense10 (Dense)              (None, 10)                3010      
Total params: 2,701,710
Trainable params: 2,701,710
Non-trainable params: 0
_________________________________________________________________


In [26]:
def get_ppd_model_hiddenperm(name='mnist', ppd=True, layerno=1):
    if name == 'mnist':
        input_shape = (28, 28, 1)
    else:
        input_shape = (32, 32, 3)  # cifar10

    def pixel2phase(images):
        img_fft = tf.fft(tf.cast(images, tf.complex64))
        phase = tf.angle(img_fft)
        return phase
    
    input_tensor = Input(name='input_images', shape=input_shape, dtype='float32')
    
    
    inner = Flatten(name='flatten')(input_tensor)
    inner = Dense(800,  name='dense800', kernel_initializer='he_normal', activation='relu')(inner)
    
    if ppd and layerno==1:
        inner = Lambda(pixel2phase, name='pixel2phase')(inner)
        #inner = Flatten(name='flatten')(inner)
        
    inner = Dense(300,  name='dense300', kernel_initializer='he_normal', activation='relu')(inner)
    
    if ppd and layerno==2:
        inner = Lambda(pixel2phase, name='pixel2phase')(inner)
    
    y_pred = Dense(10,  name='dense10', kernel_initializer='he_normal', activation='softmax')(inner)

    return Model(inputs=[input_tensor], outputs=y_pred)


In [27]:
## Phase in layer 1   (seed 87)
SECRET_SEED = 87

x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

hidden1_model = get_ppd_model_hiddenperm(name=DATASET_NAME, ppd=True, layerno=2)
hidden1_model.summary()
hidden1_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hidden1_model.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_images (InputLayer)    (None, 32, 32, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense800 (Dense)             (None, 800)               2458400   
_________________________________________________________________
dense300 (Dense)             (None, 300)               240300    
_________________________________________________________________
pixel2phase (Lambda)         (None, 300)               0         
_________________________________________________________________
dense10 (Dense)              (None, 10)                3010      
Total params: 2,701,710
Trainable params: 2,701,710
Non-trainable params: 0
_________________________________________________________________


In [28]:
## Phase in layer 2    (seed 87)
SECRET_SEED = 87

x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

hidden2_model = get_ppd_model_hiddenperm(name=DATASET_NAME, ppd=True, layerno=2)
hidden2_model.summary()
hidden2_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

hidden2_model.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_images (InputLayer)    (None, 32, 32, 3)         0         
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense800 (Dense)             (None, 800)               2458400   
_________________________________________________________________
dense300 (Dense)             (None, 300)               240300    
_________________________________________________________________
pixel2phase (Lambda)         (None, 300)               0         
_________________________________________________________________
dense10 (Dense)              (None, 10)                3010      
Total params: 2,701,710
Trainable params: 2,701,710
Non-trainable params: 0
_________________________________________________________________


In [29]:
sess =  backend.get_session()
backend.set_learning_phase(False)
attack_params = {
    'eps': 0.4
    }

def get_adv_exapmles(model, x, sess, attack_params):
    wrap = KerasModelWrapper(model)
    attack_model = FastGradientMethod(wrap, sess=sess)
    x_adv = attack_model.generate_np(x, **attack_params)
    return x_adv

x_adv_normal = get_adv_exapmles(normal_model, x_test, sess, attack_params)
x_adv_hid1 = get_adv_exapmles(hidden1_model, x_test, sess, attack_params)
x_adv_hid2 = get_adv_exapmles(hidden2_model, x_test, sess, attack_params)


[INFO 2018-12-03 20:31:09,887 cleverhans] Constructing new graph for attack FastGradientMethod
[INFO 2018-12-03 20:31:10,855 cleverhans] Constructing new graph for attack FastGradientMethod
[INFO 2018-12-03 20:31:11,377 cleverhans] Constructing new graph for attack FastGradientMethod


In [30]:
def test_model(model, x, y):
    pred = np.argmax(model.predict(x), axis = 1)
    acc =  np.mean(np.equal(pred.reshape(y.shape), y))
    return (pred, acc)


x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle_n = permute_pixels(x_adv_normal, SECRET_SEED)

pred, acc = test_model(normal_model, x_shuffle, y_test)
print ('Normal PPD MLP acc normally', acc)
pred, acc = test_model(normal_model, x_adv_shuffle_n, y_test)
print ('Normal PPD MLP acc adv', acc)




x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle_1 = permute_pixels(x_adv_hid1, SECRET_SEED)

pred, acc = test_model(hidden1_model, x_shuffle, y_test)
print ('Phase in layer 1 PPD MLP acc normally', acc)
pred, acc = test_model(hidden1_model, x_adv_shuffle_1, y_test)
print ('Phase in layer 1 PPD MLP acc adv', acc)


x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle_2 = permute_pixels(x_adv_hid2, SECRET_SEED)

pred, acc = test_model(hidden2_model, x_shuffle, y_test)
print ('Phase in layer 2 PPD MLP acc normally', acc)
pred, acc = test_model(hidden2_model, x_adv_shuffle_2, y_test)
print ('Phase in layer 2 PPD MLP acc adv', acc)



Normal PPD MLP acc normally 0.446
Normal PPD MLP acc adv 0.2383
Phase in layer 1 PPD MLP acc normally 0.4917
Phase in layer 1 PPD MLP acc adv 0.2895
Phase in layer 2 PPD MLP acc normally 0.4971
Phase in layer 2 PPD MLP acc adv 0.3289


## In CNNs: Apply pixel2phase in hidden
For mnist: 0 <= layerno <= 3

For cifar: 0 <= layerno <= 5

layerno=0 is the default ppd + cnn

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from utils import *
from sklearn.metrics import roc_curve
from keras.callbacks import EarlyStopping
from keras import backend
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils_keras import KerasModelWrapper
import matplotlib as mpl
from keras.models import load_model

DATASET_NAME = 'mnist'
(x_train, y_train, x_test, y_test) = get_dataset(DATASET_NAME) 

x_train shape (50000, 32, 32, 3)
y_train shape (50000, 1)
x_test shape (10000, 32, 32, 3)
y_test shape (10000, 1)


In [4]:
## JUST CNN

cnn_model = get_cnn_model(name=DATASET_NAME, ppd=False)
cnn_model.summary()
cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn_model.fit(x_train, y_train, epochs=25, verbose=1, batch_size=128, validation_data=(x_test, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_images (InputLayer)    (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 64)        36928     
__________

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_3/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training_1/Adam/gradients/conv2d_3/convolution_grad/Conv2DBackpropFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/gradients/conv2d_3/convolution_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_3/kernel/read)]]
	 [[{{node loss_1/mul/_241}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_997_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [4]:
## CNN + PPD, SEED 87

SECRET_SEED = 87
x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

ppd_cnn_model = get_cnn_model(name=DATASET_NAME, ppd=True)
ppd_cnn_model.summary()
ppd_cnn_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ppd_cnn_model.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_images (InputLayer)    (None, 28, 28, 1)         0         
_________________________________________________________________
pixel2phase (Lambda)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
__________

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_3/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training_1/Adam/gradients/conv2d_3/convolution_grad/Conv2DBackpropFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_1/Adam/gradients/conv2d_3/convolution_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_3/kernel/read)]]
	 [[{{node loss_1/mul/_213}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_731_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [4]:
## CNN + PPD, SEED 87

SECRET_SEED = 87
x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

ppd_cnn_model1 = get_cnn_model(name=DATASET_NAME, ppd=True, layerno=1)
ppd_cnn_model1.summary()
ppd_cnn_model1.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ppd_cnn_model1.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=32, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_images (InputLayer)    (None, 28, 28, 1)         0         
_________________________________________________________________
pixel2phase (Lambda)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
__________

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node conv2d_5/convolution}} = Conv2D[T=DT_FLOAT, _class=["loc:@training_2/Adam/gradients/conv2d_5/convolution_grad/Conv2DBackpropFilter"], data_format="NCHW", dilations=[1, 1, 1, 1], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_2/Adam/gradients/conv2d_5/convolution_grad/Conv2DBackpropFilter-0-TransposeNHWCToNCHW-LayoutOptimizer, conv2d_5/kernel/read)]]
	 [[{{node metrics_2/acc/Mean/_341}} = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_733_metrics_2/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [ ]:
## CNN + PPD, SEED 87

SECRET_SEED = 87
x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

ppd_cnn_model2 = get_cnn_model(name=DATASET_NAME, ppd=True, layerno=2)
ppd_cnn_model2.summary()
ppd_cnn_model2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ppd_cnn_model2.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

In [ ]:
## CNN + PPD, SEED 87

SECRET_SEED = 87
x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

ppd_cnn_model3 = get_cnn_model(name=DATASET_NAME, ppd=True, layerno=3)
ppd_cnn_model3.summary()
ppd_cnn_model3.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

ppd_cnn_model3.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

In [ ]:
## CNN + PPD, SEED 87
if DATASET_NAME == 'cifar10':
    SECRET_SEED = 87
    x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
    x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

    ppd_cnn_model4 = get_cnn_model(name=DATASET_NAME, ppd=True, layerno=4)
    ppd_cnn_model4.summary()
    ppd_cnn_model4.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    ppd_cnn_model4.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

In [ ]:
## CNN + PPD, SEED 87
if DATASET_NAME == 'cifar10':
    SECRET_SEED = 87
    x_train_shuffle = permute_pixels(x_train, SECRET_SEED)
    x_test_shuffle = permute_pixels(x_test, SECRET_SEED)

    ppd_cnn_model5 = get_cnn_model(name=DATASET_NAME, ppd=True, layerno=5)
    ppd_cnn_model5.summary()
    ppd_cnn_model5.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    ppd_cnn_model5.fit(x_train_shuffle, y_train, epochs=50, verbose=1, batch_size=128, validation_data=(x_test_shuffle, y_test), callbacks=[EarlyStopping(monitor='val_loss', patience=2)])

In [ ]:
## Get adv from CNN, CNN+PPD (87), CNN + Phase, CNN + Perm (87)

sess =  backend.get_session()
backend.set_learning_phase(False)
attack_params = {
    'eps': 0.4
    }

def get_adv_exapmles(model, x, sess, attack_params):
    wrap = KerasModelWrapper(model)
    attack_model = FastGradientMethod(wrap, sess=sess)
    x_adv = attack_model.generate_np(x, **attack_params)
    return x_adv

x_adv_cnn = get_adv_exapmles(cnn_model, x_test, sess, attack_params)
x_adv_ppd = get_adv_exapmles(ppd_cnn_model, x_test, sess, attack_params)

x_adv_ppd1 = get_adv_exapmles(ppd_cnn_model1, x_test, sess, attack_params)
x_adv_ppd2 = get_adv_exapmles(ppd_cnn_model2, x_test, sess, attack_params)
x_adv_ppd3 = get_adv_exapmles(ppd_cnn_model3, x_test, sess, attack_params)
if DATASET_NAME == 'cifar10':    
    x_adv_ppd4 = get_adv_exapmles(ppd_cnn_model4, x_test, sess, attack_params)
    x_adv_ppd5 = get_adv_exapmles(ppd_cnn_model5, x_test, sess, attack_params)


In [ ]:
## Test cnn_adv on cnn, test cnn+ppd (87) adv on cnn+ppd (87) and cnn+ppd (23)

def test_model(model, x, y):
    pred = np.argmax(model.predict(x), axis = 1)
    acc =  np.mean(np.equal(pred.reshape(y.shape), y))
    return (pred, acc)

pred, acc = test_model(cnn_model, x_test, y_test)
print ('CNN acc normally', acc)
pred, acc = test_model(cnn_model, x_adv_cnn, y_test)
print ('CNN acc adv', acc)

x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle = permute_pixels(x_adv_ppd, SECRET_SEED)

pred, acc = test_model(ppd_cnn_model, x_shuffle, y_test)
print ('PPD CNN acc normally', acc)
pred, acc = test_model(ppd_cnn_model, x_adv_shuffle, y_test)
print ('PPD CNN acc adv', acc)



## hidden phase
x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle = permute_pixels(x_adv_ppd1, SECRET_SEED)

pred, acc = test_model(ppd_cnn_model1, x_shuffle, y_test)
print ('PPD CNN phase_hidden = 1 acc ', acc)
pred, acc = test_model(ppd_cnn_model1, x_adv_shuffle, y_test)
print ('PPD CNN phase_hidden = 1 acc adv', acc)

x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle = permute_pixels(x_adv_ppd2, SECRET_SEED)

pred, acc = test_model(ppd_cnn_model2, x_shuffle, y_test)
print ('PPD CNN phase_hidden = 2 acc ', acc)
pred, acc = test_model(ppd_cnn_model2, x_adv_shuffle, y_test)
print ('PPD CNN phase_hidden = 2 acc adv', acc)

x_shuffle = permute_pixels(x_test, SECRET_SEED)
x_adv_shuffle = permute_pixels(x_adv_ppd3, SECRET_SEED)

pred, acc = test_model(ppd_cnn_model3, x_shuffle, y_test)
print ('PPD CNN phase_hidden = 3 acc ', acc)
pred, acc = test_model(ppd_cnn_model3, x_adv_shuffle, y_test)
print ('PPD CNN phase_hidden = 3 acc adv', acc)

if DATASET_NAME == 'cifar10':
    x_shuffle = permute_pixels(x_test, SECRET_SEED)
    x_adv_shuffle = permute_pixels(x_adv_ppd4, SECRET_SEED)

    pred, acc = test_model(ppd_cnn_model4, x_shuffle, y_test)
    print ('PPD CNN phase_hidden = 4 acc ', acc)
    pred, acc = test_model(ppd_cnn_model4, x_adv_shuffle, y_test)
    print ('PPD CNN phase_hidden = 4 acc adv', acc)

    x_shuffle = permute_pixels(x_test, SECRET_SEED)
    x_adv_shuffle = permute_pixels(x_adv_ppd5, SECRET_SEED)

    pred, acc = test_model(ppd_cnn_model5, x_shuffle, y_test)
    print ('PPD CNN phase_hidden = 5 acc ', acc)
    pred, acc = test_model(ppd_cnn_model5, x_adv_shuffle, y_test)
    print ('PPD CNN phase_hidden = 5 acc adv', acc)


